# TRANSCRIBE USING URL

In [13]:
import mysql.connector
import pandas as pd
import sqlalchemy
import pymysql
import numpy as np

In [14]:
%cd C:\Users\Admin\GENAI PROJECTS\Audio Processing

C:\Users\Admin\GENAI PROJECTS\Audio Processing


# Existing Whatsapp IB calls with call duration greater than 5 minutes

In [15]:
conn = mysql.connector.connect(
  user='nitesh.kr', password='yRpRtyJR0XSqVLibLT', host='gateway.echl.in', database='testenv_prod',port=10004)
     

cursor = conn.cursor()

query = '''SELECT agent_name,TL_name,Dept_name,did,audiofile,
'' AS transcription,'' AS sentiment,'' AS call_opening
FROM
(SELECT ccd.lead_contact_number, did,DATE(start_time) AS x1date,call_id, start_time, TIME(start_time) AS call_start_time,ccd.agentid, monitor_ucid, 
		TIMESTAMPDIFF(SECOND, start_time, end_time) call_time,TIME(end_time) AS call_end_time,ccd.type,
	IF(LOCATE('>',ccd.agentid),REVERSE(LEFT(REVERSE (ccd.agentid),LOCATE('>',REVERSE (ccd.agentid))-2)),agentid) AS 'CloudAgent_ID', 
	campaign_name,skill,TIMEDIFF(end_time,start_time) 'Time_diff',calling_status,
       	IF(cu.cloudAgentId IS NULL OR cu.cloudAgentId ='','Inbound Sales',dept_name) 'Dept_Name',
        IF(cu.cloudAgentId IS NULL OR cu.cloudAgentId ='','Abandoned_calls',cu.name) 'Agent_Name',
        IF(cu.cloudAgentId IS NULL OR cu.cloudAgentId ='','Abandoned_calls',cu1.name) 'TL_Name',
        ccd.lead_id,audiofile
        
	FROM customer_calldetails AS ccd 
	LEFT JOIN crm_users AS cu ON cu.cloudAgentId = IF(LOCATE('>',agentid),REVERSE(LEFT(REVERSE (agentid),LOCATE('>',REVERSE (agentid))-2)),agentid)
	AND cu.cloudAgentId IS NOT NULL AND cu.cloudAgentId <>''
	LEFT JOIN crm_users AS cu1 ON cu1.id = cu.reporting_to
	LEFT JOIN dept_master AS dm ON dm.id = cu.dept_id
	WHERE start_time  BETWEEN  CONCAT(CURDATE()-INTERVAL 2 DAY,' ','00:00:00') AND CONCAT(CURDATE()-INTERVAL 2 DAY,' ','23:59:59')
	##AND (LEFT(did,5) = 91124 OR LEFT(DID,4) = 9122)
	AND did IN (911244332599,911244332526,911244332582)
	##AND DATE(start_time)>= '2022-02-21'
	GROUP BY monitor_ucid
        HAVING skill NOT LIKE '%existing_booking_hindi%' AND skill NOT LIKE '%existing_booking_english%' 
        AND Time_diff >= '00:05:00')x1
'''
myc = conn.cursor()
myc.execute(query)
table_rows = myc.fetchall()
data = pd.DataFrame(table_rows)
data.columns = [i[0] for i in myc.description]
data.head()
##print(df)

,agent_name,TL_name,Dept_name,did,audiofile,transcription,sentiment,call_opening
0,Amit Kumar_1314_Gurgaon,Ankur Malik_3709_Gurgaon,Inbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,
1,Sita Patel_1224_Gurgaon,Prachi_1039_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,
2,Babli_1387_Gurgaon,Prachi_1039_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,
3,Vivek Maheshwari_3572_Gurgaon,Vivek Singh_1026_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,
4,Sita Patel_1224_Gurgaon,Prachi_1039_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,


# Taking random 5 calls from the dataframe

In [16]:
random_rows = data.sample(n=5, random_state=1)
random_rows_reset = random_rows.reset_index(drop=True)
random_rows = random_rows_reset

In [17]:
random_rows

,agent_name,TL_name,Dept_name,did,audiofile,transcription,sentiment,call_opening
0,Farhan Raza_1121_Gurgaon,Prachi_1039_Gurgaon,Outbound Sales,911244332526,https://s3.ap-southeast-1.amazonaws.com/record...,,,
1,Alisha_4469_Gurgaon,Prachi_1039_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,
2,Farhan Raza_1121_Gurgaon,Prachi_1039_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,
3,ritik_3071_gurgaon,Vivek Singh_1026_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,
4,Alisha_4469_Gurgaon,Prachi_1039_Gurgaon,Outbound Sales,911244332599,https://s3.ap-southeast-1.amazonaws.com/record...,,,


# Taking out Transcription,Sentiment & Call opening for these 5 calls

In [19]:
import os
import openai
#from openai import OpenAI
import requests


##client = OpenAI(api_key = os.environ['OPEN_API_KEY'])
##client = OpenAI()
##openai.api_key = os.getenv('OPEN_API_KEY')


#openai.api_key = 'open_API_key'
openai.api_key = 'open_API_key'

index_to_update = 0
#for index,audiofile in random_rows.iteritems():
for index,row in random_rows.iterrows():
#for index,audiofile in random_rows.iteritems():
    #url = f'{audiofile}'
    url = row['audiofile']
    response = requests.get(url)
    if response.status_code == 200:
        audio_data = response.content
    
# Save the audio file locally
    with open('audio.mp3', 'wb') as f:
        f.write(audio_data)

# Transcribe the audio using Whisper API
    with open('audio.mp3', 'rb') as audio_file:
        transcription = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file,
            language="en",
            response_format = "text"
    )
    
        response = openai.ChatCompletion.create(
          model="gpt-4o-mini",
          messages=[
            {
              "role": "user",
              "content":  "Classify the communication as positive,neutral,negative:"+transcription,               
            }
          ]
        )
        r1 = response.choices[0].message.content
        
        response = openai.ChatCompletion.create(
              model="gpt-4o-mini",
              messages=[
                {
                  "role": "user",
                  "content":  "Provide feedback on the call opening checking that all the proper behavioral traits were considered:"+transcription,               
                }
              ]
            )    
        r2 = response.choices[0].message.content
        
    
    print(transcription)
    
    print(r1)
    print(r2)
    
    
    random_rows.at[index_to_update, 'transcription'] = transcription
    random_rows.at[index_to_update, 'sentiment'] = r1
    random_rows.at[index_to_update, 'call_opening'] = r2
    index_to_update+=1


NameError: name 'audio_data' is not defined

In [ ]:
random_rows

In [ ]:
random_rows.to_csv('random_rows.csv')